## Series Codes:
https://www.bls.gov/help/hlpforma.htm#AP

In [1]:
import pandas as pd
import json
import requests

In [2]:
#First piece to get FIPS for all TN counties: odd numbers between 0 and 189:

#empty list
output = []
#range
start, end = 1, 189
#iterating each FIPS in TN counties list
for num in range(start, end +1):
    #checking condition
    if num % 2 != 0:
        print('{numb:03d}'.format(numb=num))
        #append each instance to fill list with all FIPS
        output.append('{numb:03d}'.format(numb=num))

001
003
005
007
009
011
013
015
017
019
021
023
025
027
029
031
033
035
037
039
041
043
045
047
049
051
053
055
057
059
061
063
065
067
069
071
073
075
077
079
081
083
085
087
089
091
093
095
097
099
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189


In [3]:
#Create full list of all FIPS:

#create empty list
FIPS = []
#for each number in output
for n in output:
    print('CN47{}00000000'.format(n))
    #append to fill list
    FIPS.append('CN47{}00000000'.format(n))

CN4700100000000
CN4700300000000
CN4700500000000
CN4700700000000
CN4700900000000
CN4701100000000
CN4701300000000
CN4701500000000
CN4701700000000
CN4701900000000
CN4702100000000
CN4702300000000
CN4702500000000
CN4702700000000
CN4702900000000
CN4703100000000
CN4703300000000
CN4703500000000
CN4703700000000
CN4703900000000
CN4704100000000
CN4704300000000
CN4704500000000
CN4704700000000
CN4704900000000
CN4705100000000
CN4705300000000
CN4705500000000
CN4705700000000
CN4705900000000
CN4706100000000
CN4706300000000
CN4706500000000
CN4706700000000
CN4706900000000
CN4707100000000
CN4707300000000
CN4707500000000
CN4707700000000
CN4707900000000
CN4708100000000
CN4708300000000
CN4708500000000
CN4708700000000
CN4708900000000
CN4709100000000
CN4709300000000
CN4709500000000
CN4709700000000
CN4709900000000
CN4710100000000
CN4710300000000
CN4710500000000
CN4710700000000
CN4710900000000
CN4711100000000
CN4711300000000
CN4711500000000
CN4711700000000
CN4711900000000
CN4712100000000
CN4712300000000
CN471250

In [4]:
#create a function - NOTE I changed v1 to v2... in the request post, if it gets touchy change back I guess

def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,"startyear":"%d" % (start), "endyear":"%d" % (end)})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])


start = 2014
end = 2018
series = ['LAUCN281070000000003']

df = get_bls_data(series=series, start=start, end=end)

In [6]:
df.head()

,year,period,periodName,value,footnotes,seriesID
0,2018,M12,December,6.1,,LAUCN281070000000003
1,2018,M11,November,5.4,,LAUCN281070000000003
2,2018,M10,October,5.5,,LAUCN281070000000003
3,2018,M09,September,5.6,,LAUCN281070000000003
4,2018,M08,August,6.2,,LAUCN281070000000003


In [7]:
#check if there are unique values in footnotes, if not drop the column
df['footnotes'].value_counts()
df = df.drop(columns = 'footnotes')

In [8]:
#pull out county FIPS to join to gdb
df['FIPS'] = df['seriesID'].str.slice(5, 10)
df.head()

,year,period,periodName,value,seriesID,FIPS
0,2018,M12,December,6.1,LAUCN281070000000003,28107
1,2018,M11,November,5.4,LAUCN281070000000003,28107
2,2018,M10,October,5.5,LAUCN281070000000003,28107
3,2018,M09,September,5.6,LAUCN281070000000003,28107
4,2018,M08,August,6.2,LAUCN281070000000003,28107


In [9]:
df.to_csv('../outputs/tncounties_ue.csv', index=False)